Import all necessary libraries.

In [2]:
!pip install thefuzz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from thefuzz import process
import re

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## **Experiment Merging using Fuzzy Logic**

Try to merge datasets using Fuzzy logic, based on ingredient names and their similarity. The purpose of merging is to extend ingredients with macro values.

In [5]:
# Load the datasets
recipes_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/Exploded_Recipes.csv')
food_macro_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/FoodMacroDetailedInfo.csv')

# Normalize text data to lower case to ensure case-insensitive matching
recipes_df['ingredient'] = recipes_df['ingredient'].str.lower()
food_macro_df['Food Name'] = food_macro_df['Food Name'].str.lower()

# Prepare an empty DataFrame to store the results
enriched_recipes = pd.DataFrame()

# Function to find the best match using thefuzz
def find_best_match(ingredient):
    # Find the best match and its score
    best_match, score = process.extractOne(ingredient, food_macro_df['Food Name'].tolist(), scorer=process.fuzz.token_sort_ratio)
    # Only consider the match if the score is high enough
    if score > 75:  # Adjust the threshold as needed
        return best_match
    return None

# Apply the matching function to each ingredient and enrich the recipes
for index, row in recipes_df.iterrows():
    ingredient = row['ingredient']
    best_match = find_best_match(ingredient)
    if best_match:
        match_row = food_macro_df[food_macro_df['Food Name'] == best_match].iloc[0]
        enriched_row = row.copy()
        for col in food_macro_df.columns:
            enriched_row[col] = match_row[col]
        enriched_recipes = pd.concat([enriched_recipes, pd.DataFrame([enriched_row])], ignore_index=True)

# Optionally save the merged data to a new CSV file
enriched_recipes.to_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/enriched_recipesxx.csv', index=False)

# Print a sample of the enriched dataset to verify
print(enriched_recipes.head())


                     food_title  \
0     Crunchy Onion Potato Bake   
1       Spinach and Yogurt Soup   
2       Spinach and Yogurt Soup   
3  Peanut Butter Fudge Frosting   
4                   Mostaccioli   

                                        instructions  \
0  Preheat oven to 350 degrees Fahrenheit. /t Spr...   
1  Wash and drain spinach; remove any large stems...   
2  Wash and drain spinach; remove any large stems...   
3  In a bowl, using an electric mixer, beat toget...   
4  In bowl, beat butter with granulated sugar unt...   

                            ingredient       ID                 Food Name  \
0                 1 cup cheddar cheese   328637           cheese, cheddar   
1  1 tablespoon extra virgin olive oil   748608  oil, olive, extra virgin   
2        2 14 cups plain nonfat yogurt  2647437     yogurt, plain, nonfat   
3               34 cup unsalted butter   789828   butter, stick, unsalted   
4               14 cup unsalted butter   789828   butter, stick, un

Order this merged dataset.

In [6]:
# Load the enriched recipes file
enriched_recipes_path = '/content/drive/MyDrive/Colab Notebooks/DS Project food/enriched_recipesxx.csv'
enriched_recipes_df = pd.read_csv(enriched_recipes_path)

# Sort the DataFrame by the 'food_title' column alphabetically
enriched_recipes_df_sorted = enriched_recipes_df.sort_values(by='food_title')

# Save the sorted DataFrame back to a new CSV file
sorted_recipes_path = '/content/drive/MyDrive/Colab Notebooks/DS Project food/sorted_enriched_recipesxx.csv'
enriched_recipes_df_sorted.to_csv(sorted_recipes_path, index=False)

# Print a sample of the sorted dataset to verify
print(enriched_recipes_df_sorted.head())

                             food_title  \
7759            "Hold-the-Phone" Burger   
8820                "Paella" Fried Rice   
6853  "Rye"-Raisin Belgian Waffle Bread   
5272         10 Minute Szechuan Chicken   
6149  15 Minute Chicken and Rice Dinner   

                                           instructions  \
7759  Shape meat loosely into 12 (6-oz./170-g) balls...   
8820  Heat oil in a 12-inch nonstick skillet over mo...   
6853  In a large bowl, whisk sorghum flour with mill...   
5272  Cut chicken into 1 1/2 inch cubes. /t Lightly ...   
6149  Brown chicken in hot oil, 5 minutes on each si...   

                                     ingredient       ID  \
7759               3/4 cup creamy peanut butter  2262072   
8820             1/4 cup extra-virgin olive oil   748608   
6853                      13 c. unsalted butter   789828   
5272  4 boneless skinless chicken breast halves  2646170   
6149        4 boneless skinless chicken breasts  2646170   

                         

Note, there are a lot of missing ingredients due to few similarities between food data in the Macro dataset and food data in the ingredients column.

Try to merge again using fuzzy logic and common mappings for food names.

In [8]:
# Load the datasets
recipes_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/Exploded_Recipes.csv')
food_macro_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/FoodMacroDetailedInfo.csv')

# Normalize text data to lower case to ensure case-insensitive matching
recipes_df['ingredient'] = recipes_df['ingredient'].str.lower().str.strip()
food_macro_df['Food Name'] = food_macro_df['Food Name'].str.lower().str.strip()

# Predefined mapping for common ingredients (50 mappings)
common_mappings = {
    'milk': 'milk, lowfat, fluid, 1% milkfat, with added vitamin a and vitamin d',
    'ketchup': 'ketchup, no salt added',
    'sugar': 'sugar, granulated',
    'flour': 'wheat flour, white, all-purpose, enriched, bleached',
    'butter': 'butter, salted',
    'egg': 'egg, whole, raw, fresh',
    'onion': 'onions, raw',
    'garlic': 'garlic, raw',
    'salt': 'salt, table',
    'pepper': 'spices, pepper, black',
    'olive oil': 'oil, olive, salad or cooking',
    'chicken': 'chicken, broilers or fryers, meat only, raw',
    'beef': 'beef, ground, 85% lean meat / 15% fat, raw',
    'carrot': 'carrots, raw',
    'potato': 'potatoes, flesh and skin, raw',
    'tomato': 'tomatoes, red, ripe, raw, year round average',
    'cheese': 'cheese, cheddar',
    'bread': 'bread, whole-wheat',
    'rice': 'rice, white, long-grain, regular, raw, unenriched',
    'pasta': 'pasta, cooked, unenriched, without added salt',
    'banana': 'bananas, raw',
    'apple': 'apples, raw, with skin',
    'lettuce': 'lettuce, raw',
    'spinach': 'spinach, raw',
    'broccoli': 'broccoli, raw',
    'cucumber': 'cucumbers, raw, with peel',
    'corn': 'corn, sweet, yellow, raw',
    'mushroom': 'mushrooms, white, raw',
    'cabbage': 'cabbage, raw',
    'peanut butter': 'peanut butter, smooth style, with salt',
    'yogurt': 'yogurt, plain, whole milk',
    'honey': 'honey',
    'lemon': 'lemons, raw, without peel',
    'lime': 'limes, raw',
    'orange': 'oranges, raw, all commercial varieties',
    'grape': 'grapes, raw',
    'strawberry': 'strawberries, raw',
    'blueberry': 'blueberries, raw',
    'raspberry': 'raspberries, raw',
    'blackberry': 'blackberries, raw',
    'pineapple': 'pineapple, raw, all varieties',
    'watermelon': 'watermelon, raw',
    'cantaloupe': 'melons, cantaloupe, raw',
    'peach': 'peaches, raw',
    'pear': 'pears, raw',
    'plum': 'plums, raw',
    'cherry': 'cherries, sweet, raw',
    'kiwi': 'kiwifruit, green, raw',
    'mango': 'mangos, raw',
    'papaya': 'papayas, raw',
    'buttermilk': 'buttermilk, fluid, whole',
    'mayonnaise': 'mayonnaise, regular',
    'soy sauce': 'soy sauce made from soy (tamari)',
    'vinegar': 'vinegar, distilled',
    'cream': 'cream, fluid, heavy whipping',
    'coconut milk': 'beverage, coconut milk, unsweetened',
    'chocolate': 'chocolate, dark, 70-85% cacao solids',
    'raisins': 'raisins, seedless',
    'almonds': 'nuts, almonds',
    'walnuts': 'nuts, walnuts, english',
    'pecans': 'nuts, pecans',
    'cashews': 'nuts, cashew nuts, raw',
    'hazelnuts': 'nuts, hazelnuts or filberts',
    'peanut': 'nuts, peanuts, all types, raw',
    'sunflower seeds': 'seeds, sunflower seed kernels, dried',
    'sesame seeds': 'seeds, sesame seeds, whole, dried',
    'chia seeds': 'seeds, chia seeds, dried',
    'flax seeds': 'seeds, flaxseed',
    'pistachios': 'nuts, pistachio nuts, raw',
    'dates': 'dates, medjool',
    'figs': 'figs, raw',
    'prunes': 'prunes, dried (pitted)',
    'apricots': 'apricots, raw',
    'peppers': 'peppers, sweet, green, raw',
    'zucchini': 'squash, summer, zucchini, includes skin, raw',
    'eggplant': 'eggplant, raw',
    'pumpkin': 'pumpkin, raw',
    'acorn squash': 'squash, winter, acorn, raw',
    'butternut squash': 'squash, winter, butternut, raw',
    'cauliflower': 'cauliflower, raw',
    'green beans': 'beans, snap, green, raw',
    'chickpeas': 'chickpeas (garbanzo beans, bengal gram), mature seeds, raw',
    'lentils': 'lentils, raw',
    'kidney beans': 'beans, kidney, all types, mature seeds, raw',
    'black beans': 'beans, black turtle, mature seeds, raw',
    'navy beans': 'beans, navy, mature seeds, raw',
    'pinto beans': 'beans, pinto, mature seeds, raw'
}

# Function to find the best match using thefuzz
def find_best_match(ingredient):
    if ingredient in common_mappings:
        return common_mappings[ingredient]
    best_match, score = process.extractOne(ingredient, food_macro_df['Food Name'].tolist(), scorer=process.fuzz.token_sort_ratio)
    if score > 75:  # Adjust the threshold as needed
        return best_match
    return None

# Create a new DataFrame to hold the matches and macronutrient information
matches = []

# Iterate through each ingredient in recipes_df
for index, row in recipes_df.iterrows():
    ingredient = row['ingredient']
    best_match = find_best_match(ingredient)
    match_info = {'ingredient': ingredient}
    if best_match:
        match_row = food_macro_df[food_macro_df['Food Name'] == best_match]
        if not match_row.empty:
            match_row = match_row.iloc[0]
            for col in food_macro_df.columns:
                match_info[col] = match_row[col]
    matches.append(match_info)

# Create a DataFrame from the matches
matches_df = pd.DataFrame(matches)

# Merge the matches with the original recipes DataFrame
enriched_recipes = pd.concat([recipes_df.reset_index(drop=True), matches_df.drop(columns=['ingredient'])], axis=1)

# Optionally save the merged data to a new CSV file
enriched_recipes.to_csv('/content/drive/MyDrive/Colab Notebooks/DS Project food/xd.csv', index=False)

# Print a sample of the enriched dataset to verify
print(enriched_recipes.head())


                  food_title  \
0  Crunchy Onion Potato Bake   
1  Crunchy Onion Potato Bake   
2  Crunchy Onion Potato Bake   
3  Crunchy Onion Potato Bake   
4  Crunchy Onion Potato Bake   

                                        instructions  \
0  Preheat oven to 350 degrees Fahrenheit. /t Spr...   
1  Preheat oven to 350 degrees Fahrenheit. /t Spr...   
2  Preheat oven to 350 degrees Fahrenheit. /t Spr...   
3  Preheat oven to 350 degrees Fahrenheit. /t Spr...   
4  Preheat oven to 350 degrees Fahrenheit. /t Spr...   

                                    ingredient  ID Food Name  Kcal  Protein  \
0                               2 12 cups milk NaN       NaN   NaN      NaN   
1                              1 12 cups water NaN       NaN   NaN      NaN   
2                                14 cup butter NaN       NaN   NaN      NaN   
3            mashed potatoes, 1 box, homestyle NaN       NaN   NaN      NaN   
4  1 (8 ounce) can whole kernel corn (drained) NaN       NaN   NaN      NaN

**Sadly, the merge is still impossible due to few parallels between the columns. The 1M recipe ingredients are very badly web scraped when the dataset was made, thus there is no common guideline and there are way too many inconsistences for merging to be possible with any common logic.**

Create a merge where all word instances in the ingredient dataset match all words instances for the food names in the Macro dataset. (This is not usable, but experimental). The point is to view how many items can be merged and to see if words can actually be merged in these datasets.

In [9]:
# Function to normalize strings
def normalize_string(s):
    s = s.lower()  # Convert to lowercase
    s = re.sub(r'[^a-zA-Z0-9\s]', '', s)  # Remove punctuation
    s = s.strip()  # Trim whitespace
    return s

# Function to extract unique words from a string
def extract_words(s):
    return set(s.split())

# Load the datasets
recipes_path = '/content/drive/MyDrive/Colab Notebooks/DS Project food/Reduced_Recipes.csv'
macros_path = '/content/drive/MyDrive/Colab Notebooks/DS Project food/FoodMacroDetailedInfo.csv'

recipes_df = pd.read_csv(recipes_path)
macros_df = pd.read_csv(macros_path)

# Normalize the Ingredient and Food Name columns
recipes_df['ingredient_normalized'] = recipes_df['ingredient'].apply(normalize_string)
macros_df['food_name_normalized'] = macros_df['Food Name'].apply(normalize_string)

# Extract words from the normalized columns
recipes_df['ingredient_words'] = recipes_df['ingredient_normalized'].apply(extract_words)
macros_df['food_name_words'] = macros_df['food_name_normalized'].apply(extract_words)

# Function to check if there is any common word between two sets
def has_common_word(set1, set2):
    return not set1.isdisjoint(set2)

# Create a list to hold the matches
matches = []

# Find matches based on common words
for i, recipe_row in recipes_df.iterrows():
    for j, macro_row in macros_df.iterrows():
        if has_common_word(recipe_row['ingredient_words'], macro_row['food_name_words']):
            match = {**recipe_row, **macro_row}
            matches.append(match)

# Create a DataFrame from the matches
merged_df = pd.DataFrame(matches)

# Save the merged dataset to a new CSV file
merged_file_path = '/content/drive/MyDrive/Colab Notebooks/DS Project food/Merged_Recipes_AllNames_Macros.csv'
merged_df.to_csv(merged_file_path, index=False)

print(f"Merged dataset saved to {merged_file_path}")


Merged dataset saved to /content/drive/MyDrive/Colab Notebooks/DS Project food/Merged_Recipes_AllNames_Macros.csv
